In [1]:
import requests
import sqlite3
import pandas as pd
import datetime
from IPython.display import display

Создадим базу данных itunes.db:

In [2]:
connection = sqlite3.connect('itunes.db')
cur = connection.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS itunes(
   id INT,
   word TEXT,
   pos INT,
   date DATETIME);
""")
connection.commit()

Запишем необходимые функции (поиска полного названия по id) и поиска позиции по словам названия:

In [3]:
def lookup(object_id):
    url = "https://itunes.apple.com/lookup"
    payload = {"id": object_id}
    response = requests.get(url, params=payload)
    response.raise_for_status()
    results = response.json().get("results", [])
    return results


def search(term, media="software", limit=200):
    url = "https://itunes.apple.com/search"
    payload = {"term": term, "media": media, "limit": limit}
    response = requests.get(url, params=payload)
    response.raise_for_status()
    results = response.json().get("results", [])
    return results

def insert_results_into_db(final_table):
    cur.executemany("INSERT INTO itunes VALUES(?, ?, ?, ?);", final_table)
    connection.commit()
    
def find_position_and_save_res(object_id):
    lookup_result=lookup(object_id)
    found_name=lookup_result[0]["trackName"]
    words=found_name.split()
    
    final_table=[]
    cur_date=datetime.datetime.now()
    position=0

    for word in words:
        each_search=search(word)
        for k in range(len(each_search)):
            if each_search[k]['trackName']==found_name:
                position=k+1
                break
        
            else:
                position=None
    
        final_table.append([object_id, word, position, cur_date])
    
    insert_results_into_db(final_table)

Вызовем функцию для приложения с id=860011430:

In [6]:
find_position_and_save_res(860011430)

cur.execute("SELECT * FROM itunes;")
many_res = cur.fetchall()

df=pd.DataFrame(columns=['id','word', 'pos', 'date'], data=many_res)
display(df)

,id,word,pos,date
0,860011430,Ghost,9.0,2021-07-19 22:05:40.108942
1,860011430,Lens,6.0,2021-07-19 22:05:40.108942
2,860011430,AR,NaN,2021-07-19 22:05:40.108942
3,860011430,Fun,NaN,2021-07-19 22:05:40.108942
4,860011430,Movie,36.0,2021-07-19 22:05:40.108942
...,...,...,...,...
79,860011430,Lens,6.0,2021-07-20 08:32:43.390978
80,860011430,AR,NaN,2021-07-20 08:32:43.390978
81,860011430,Fun,NaN,2021-07-20 08:32:43.390978
82,860011430,Movie,36.0,2021-07-20 08:32:43.390978
